# Silver - Schema enforcment, data cleaning and validation
# Purpose: Establish functions that clean bronze property table
# Source: unity catalog table path
# Output: functions
## - transform_property_bronze_silver("unity_catalog path")
## -  property_run_uc_tests("unity_catalog path") 

## CONFIG/PARAMETERS


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
import time

In [0]:
dbutils.widgets.text("year", "")
year = dbutils.widgets.get("year")

In [0]:
notebook_name = "property_silver"
stage = "silver"
input_table = "bronze_property"
output_table = "validation_silver_property"
test_table = output_table

state_class_map = {
    "a1": "single_family_residential"
    , "a2": "mobile_home"
    # , "b1": "multi_family_residential"
}

allowed_state_classes = ", ".join(f"'{s}'" for s in state_class_map.values())

Imports logging and metric functions

In [0]:
%run ../log_notebook

## TRANSFORMATIONS

necessary columns from source table
- 

In [0]:
def schema_check(data_frame):
    # all NECCESSARY COLUMNS from source
    expected_columns = {"acct", "site_addr_1", "site_addr_2", "site_addr_3", "state_class", "bld_ar", "land_ar", "tot_mkt_val", "yr"}
    missing = expected_columns - set(data_frame.columns)
    if missing:
        missing_string = ", ".join(missing)
        print(f"ERROR: assumption failed - {missing_string} missing columns")
        raise ValueError(f"Missing columns: {missing_string}")
    

In [0]:
def transform_property_bronze_silver(data_frame):

    log_pipeline_runs(
        notebook=notebook_name, stage=stage, input_table=input_table, output_table=output_table, status="START"
        )
    
    start_time = time.monotonic()
    
    try:
        print("start transformation zip from bronze to silver")

        print("start checking assumptions")
        ## ASUMPTION CHECKING

        # TABLE has NECESSARY COLUMNS
        schema_check(data_frame)

        print("end checking assumptions")
        print("start transformations")

        # CALCULATION amount of ROWS in bronze table
        bronze_rows = data_frame.count()
        print(f"amount of rows in bronze table: {bronze_rows}")

        ## TRANSFORMATION
        # LOWER, TRIM, RENAME, AND CAST
        # FILL N/A AND REPLACE malformed values

        property_renamed = (
            data_frame
            .fillna("unknown", subset=["acct", "site_addr_1", "site_addr_2", "state_class"])
            .fillna("0", subset=["bld_ar", "land_ar", "tot_mkt_val"])
            .fillna("00000", subset=["site_addr_3"]) 
            .select(
                F.lower(F.trim("acct")).alias("dim_account_number").cast("string")
                , F.lower(F.trim("site_addr_1")).alias("dim_street").cast("string")
                , F.lower(F.trim("site_addr_2")).alias("dim_city").cast("string")
                , F.lower(F.trim("site_addr_3")).alias("dim_zip_code").cast("string")
                , F.lower(F.trim("state_class")).alias("dim_state_class").cast("string")
                , F.lower(F.trim("bld_ar")).alias("m_building_area").cast("bigint")
                , F.lower(F.trim("land_ar")).alias("m_land_area").cast("bigint")
                , F.lower(F.trim("tot_mkt_val")).alias("m_total_market_value").cast("bigint")
                , F.to_date(F.trim(F.col("yr")), "yyyy").alias("dim_year_date")
            )
            .replace({"":"unknown"}, subset = ["dim_account_number", "dim_street", "dim_city", "dim_state_class"])
            # .replace({0:0}, subset=["m_building_area", "m_land_area", "m_total_market_value"])
            .replace({"":"00000"}, subset=["dim_zip_code"])
        )
        
        # CHOOSE VALID state_classes, account numbres and zip codes COLUMNS 
        # RENAME state_classes
        # FIX FORMAT dim_street
        # (state clasess chosen in parameters, account numbers that are not null, zip codes with 5 values) 
        property_valid = (
            property_renamed
            .where(
                (F.col("dim_state_class").isin(list(state_class_map.keys())))
                & (F.col("dim_account_number").isNotNull())
                & (F.col("dim_zip_code").rlike("^[0-9]{5}$"))
                # & ~(F.col("m_total_market_value") == 0), enforce business rules later
                )
            .replace(state_class_map, subset=["dim_state_class"])
            .withColumn("dim_street", F.regexp_replace(F.col("dim_street"), " ", "_"))
        )
        
        ## METRIC CALCULATIONS
        # METRICS: 
        # - rows with null in dim_street
        # - rows with market value of 0
        # - rows with zip code of "00000"
        # - duration of tranformation
        # - number of rows post transformation

        property_valid.createOrReplaceTempView("temp_property_valid")

        metrics = spark.sql(f"""
            SELECT 
                COUNT(*) as silver_rows
                , COUNT(
                    CASE WHEN
                        dim_street = 'unknown'
                        THEN 1
                    END
                ) as empty_street_rows
                , COUNT(
                    CASE WHEN
                        m_total_market_value = 0
                        THEN 1
                    END
                ) as empty_market_value_rows
                , COUNT(
                    CASE WHEN
                        dim_zip_code = '00000'
                        THEN 1
                    END
                ) as invalid_zip_codes
            FROM temp_property_valid
        """)

        end_time = time.monotonic()
        run_time = end_time - start_time

        # CALCULATION of ROWS in post TRANSFORMATION table
        print(f"amount of rows after dropping invalid state classes, invalid zips, market values at 0, and null account number: {metric_first_row["silver_rows"]}")

        # METRIC AGGREGATION
        metric_first_row = metrics.first()
        
        metric_value_list = [bronze_rows]

        for columns in metrics.columns:
            metric_value_list.append(metric_first_row[columns])

        metric_value_list.append(run_time)

        print(metric_value_list)

        metric_name_list = ["raw_rows", "clean_rows", "empty_street_rows", "empty_market_value_rows", "invalid_zips", "run_time"] #rows with 000000 in zipcode

        metric_table_upload(notebook=notebook_name, stage=stage, metric_name_list=metric_name_list, metric_value_list=metric_value_list)

        log_pipeline_runs(
            notebook=notebook_name, stage=stage, input_table=input_table, output_table=output_table, status="SUCCESS", run_time=run_time
        )
        
        return property_valid
    
    except Exception as e:
        # FAIILURE

        end_time = time.monotonic()
        run_time = end_time - start_time
        
        log_pipeline_runs(
            notebook=notebook_name, stage=stage, input_table=input_table, output_table=output_table, status="FAILURE", error_message = str(e), run_time=run_time
            )
        
        raise

## VALIDATION 

Ensures data quality
- at least 1,000,000 (arbitrary) rows with account number, zip code, state class, city, and market value
- no duplicate account number
- no null account number
- all state classes are among the selected in parameters
- property value is between 0 and 100,000,000 (upper range congruent when examining single family duelings)

In [0]:
def property_run_uc_tests(table_name: str):
    failures = []

    ## TESTS DEFINITIONS (shorts descriptions above)
    # tests made so if a data frame has more than 0 rows, then there is a data quality issue (except first)
    tests = {
        "row_count_minimum_failed":  
            f"""
            SELECT
                COUNT(*) 
            FROM {table_name} 
            WHERE dim_account_number != 'unknown' 
                AND dim_zip_code != '00000' 
                AND dim_state_class != 'unknown' 
                AND dim_city != 'unknown' 
                AND m_total_market_value != 0
            HAVING COUNT(*) < 1000000
            """
        , "duplicate_dim_account_number": 
            f"""
            SELECT 
                dim_account_number 
            FROM {table_name} 
            GROUP BY dim_account_number 
            HAVING COUNT(*) > 1 LIMIT 1
            """
        , "null_dim_account_number": f"""
            SELECT 
                dim_account_number 
            FROM {table_name} 
            WHERE dim_account_number IS NULL 
            LIMIT 1
        """
        # , "correct_length_zip": f"""
        #     SELECT 
        #         dim_zip_code 
        #     FROM {table_name} 
        #     WHERE NOT dim_zip_code REGEXP '^[0-9]{5}$'
        #     LIMIT 1
        # """
        , "correct_state_names": f"""
            SELECT 
                dim_state_class 
            FROM {table_name} 
            WHERE dim_state_class NOT IN ({allowed_state_classes})
            LIMIT 1
        """
        , "property_value_not_in_range": f"""
            SELECT 
                m_total_market_value
            FROM {table_name} 
            WHERE m_total_market_value NOT BETWEEN 0 AND 100000000
            LIMIT 1
        """
    }

    ## RUN TESTS
    for name, sql_tests in tests.items():
        if spark.sql(sql_tests).count() > 0:
            failures.append(name)
            log_data_quality_tests(notebook=notebook_name, test_table=test_table, test_name=name, status="FAILURE")
        else:
            print(f"test {name} passed")
            log_data_quality_tests(notebook=notebook_name, test_table=test_table, test_name=name, status="SUCCESS")
    
    if failures:
        raise RuntimeError(
            f"UC tests failed for {table_name}: {', '.join(failures)}"
        )

## DEBUGGING

In [0]:
%skip
input_path = f"bronze.bronze_property_{year}"
property_h_c = spark.read.table(input_path)

In [0]:
%skip
%sql
use CATALOG harris_county_catalog;

In [0]:
%skip
df = transform_property_bronze_silver(property_h_c)
df.show()

In [0]:
%skip
property_run_uc_tests("validation.property_validate")